## Notebook Overview.

**Initial Setup:** 4/20/2019

This notebook tracks the performance of a Robo Advisor-like ETF strategy and compares relative to a benchmark; for purposes of this review, we compare performance against the S&P 500.

Key sections of this notebook include the following:

1. Performance of ETF components and overall strategy.
1. Comparison relative to SP500.
1. TSR by comparing appreciation plus dividends versus SP500.

**Helpful References:**

1. [FactSet smart beta article](https://insight.factset.com/finally-smart-beta-etfs-gain-market-share)
1. [Smart Beta ETF db](https://etfdb.com/themes/smart-beta-etfs/)
1. [Personal Capital Historical Performance](https://www.personalcapital.com/wealth-management/performance)
1. SPY data returns matches Morningstar price [here](http://performance.morningstar.com/funds/etf/total-returns.action?t=spy)
1. [Wikipedia SP500 historical returns](https://en.wikipedia.org/wiki/S%26P_500_Index)
1. [Finance Python trading DataCamp blog post](https://www.datacamp.com/community/tutorials/finance-python-trading)
1. GitHub notebook for DataCamp tutorial can be found [here](https://github.com/datacamp/datacamp-community-tutorials/blob/master/Python%20Finance%20Tutorial%20For%20Beginners/Python%20For%20Finance%20Beginners%20Tutorial.ipynb)
1. Fix Yahoo Finance package [here](https://pypi.python.org/pypi/fix-yahoo-finance)

In [43]:
# Import initial libraries

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import quandl
%matplotlib inline

In [44]:
# Imports in order to be able to use Plotly offline.
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import plotly.graph_objs as go

print(__version__) # requires version >= 1.9.0

init_notebook_mode(connected=True)

5.6.0


In [45]:
# Date Ranges for SP 500 and for all tickers
# Modify these date ranges each week.

# The below will pull back stock prices from 2010 until end date specified.
start_sp = datetime.datetime(2010, 1, 1)
end_sp = datetime.datetime(2019, 4, 18)

# This variable is used for YTD performance.
end_of_last_year = datetime.datetime(2018, 12, 31)

# These are separated out if for some reason want different date range than SP.
stocks_start = datetime.datetime(2010, 1, 1)
# stocks_end = datetime.datetime(2018, 12, 10)
stocks_end = end_sp

## Data Extraction

### SP 500

In [46]:
# Leveraged from the helpful Datacamp Python Finance trading blog post.

from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

sp500 = pdr.get_data_yahoo('^GSPC', 
                           start_sp,
                             end_sp+datetime.timedelta(days=1))
                          
sp500.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-12-31,1126.599976,1127.640015,1114.810059,1115.099976,1115.099976,2076990000
2010-01-04,1116.560059,1133.869995,1116.560059,1132.989990,1132.989990,3991400000
2010-01-05,1132.660034,1136.630005,1129.660034,1136.520020,1136.520020,2491020000
2010-01-06,1135.709961,1139.189941,1133.949951,1137.140015,1137.140015,4972660000
2010-01-07,1136.270020,1142.459961,1131.319946,1141.689941,1141.689941,5270680000


In [47]:
sp500.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-04-12,2900.860107,2910.540039,2898.370117,2907.409912,2907.409912,3688490000
2019-04-15,2908.320068,2909.600098,2896.479980,2905.580078,2905.580078,3088330000
2019-04-16,2912.260010,2916.060059,2900.709961,2907.060059,2907.060059,3402210000
2019-04-17,2916.040039,2918.000000,2895.449951,2900.449951,2900.449951,3602300000
2019-04-18,2904.810059,2908.399902,2891.899902,2905.030029,2905.030029,3506850000


In [48]:
# Confirmed that these returns are excluding dividends.

sp500_annual = sp500[['Adj Close']]

sp500_annual = sp500_annual.resample(rule='A').last()

sp500_annual['YoY'] = sp500_annual.pct_change()

sp500_annual

,Adj Close,YoY
Date,,
2009-12-31,1115.099976,NaN
2010-12-31,1257.640015,0.127827
2011-12-31,1257.599976,-0.000032
2012-12-31,1426.189941,0.134057
2013-12-31,1848.359985,0.296012
2014-12-31,2058.899902,0.113906
2015-12-31,2043.939941,-0.007266
2016-12-31,2238.830078,0.095350
2017-12-31,2673.610107,0.194200


In [49]:
# Create a dataframe with only the Adj Close column as that's all we need for this analysis.

sp_500_adj_close = sp500[['Adj Close']].reset_index()

In [50]:
# Adj Close for the EOY in 2018 in order to run comparisons versus stocks YTD performances.

sp_500_adj_close_start = sp_500_adj_close[sp_500_adj_close['Date']==end_of_last_year]
sp_500_adj_close_start

,Date,Adj Close
2264,2018-12-31,2506.850098


### Robo Advisor Benchmark Portfolio

In [51]:
# Import the Sample worksheet with acquisition dates and initial cost basis:

portfolio_df = pd.read_excel('Robo Advisor Acquisition Costs.xlsx', sheet_name='Main', index_col=None)

portfolio_df.sort_values(by=['Ticker', 'Acquisition Date'], ascending=[True, True], inplace=True)

portfolio_df.reset_index(drop=True, inplace=True)

portfolio_df['Counts'] = portfolio_df.index

portfolio_df['Ticker #'] = portfolio_df['Ticker'].map(str) + ' ' + portfolio_df['Counts'].map(str)

portfolio_df

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,Ticker #
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,AGG 0
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,DBC 1
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,IAU 2
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,IGOV 3
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,VEU 4
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,VNQ 5
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,VTI 6
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,VTI 7


In [52]:
portfolio_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Acquisition Date  8 non-null      datetime64[ns]
 1   Position          8 non-null      object        
 2   Ticker            8 non-null      object        
 3   Quantity          8 non-null      int64         
 4   Unit Cost         8 non-null      float64       
 5   Cost Basis        8 non-null      float64       
 6   Start of Year     8 non-null      datetime64[ns]
 7   Target_Alloc      8 non-null      float64       
 8   Comments          0 non-null      float64       
 9   Counts            8 non-null      int64         
 10  Ticker #          8 non-null      object        
dtypes: datetime64[ns](2), float64(4), int64(2), object(3)
memory usage: 832.0+ bytes


In [53]:
# Generate a dynamic list of tickers to pull from Yahoo Finance API based on the imported file with tickers.
tickers = portfolio_df['Ticker'].unique()
tickers

array(['AGG', 'DBC', 'IAU', 'IGOV', 'VEU', 'VNQ', 'VTI'], dtype=object)

In [54]:
# Stock comparison code

def get(tickers, startdate, enddate):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate+datetime.timedelta(days=1)))
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))
               
all_data = get(tickers, stocks_start, stocks_end)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [55]:
all_data.tail()

Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
VTI    2019-04-12  148.550003  148.800003  148.130005  148.679993  141.773788   
       2019-04-15  148.750000  148.800003  148.059998  148.559998  141.659348   
       2019-04-16  149.000000  149.059998  148.240005  148.600006  141.697495   
       2019-04-17  149.199997  149.220001  147.770004  148.050003  141.173019   
       2019-04-18  148.399994  148.399994  147.509995  148.270004  141.382843   

                    Volume  
Ticker Date                 
VTI    2019-04-12  2612000  
       2019-04-15  2083000  
       2019-04-16  2953300  
       2019-04-17  2983900  
       2019-04-18  2457000

In [56]:
# Also only pulling the ticker, date and adj. close columns for our tickers.

adj_close = all_data[['Adj Close']].reset_index()
adj_close.head()

,Ticker,Date,Adj Close
0,AGG,2009-12-31,75.122749
1,AGG,2010-01-04,75.210129
2,AGG,2010-01-05,75.552269
3,AGG,2010-01-06,75.508575
4,AGG,2010-01-07,75.421249


In [57]:
# Grabbing the ticker close from the end of last year 
adj_close_start = adj_close[adj_close['Date']==end_of_last_year]
adj_close_start

,Ticker,Date,Adj Close
2264,AGG,2018-12-31,99.191551
4604,DBC,2018-12-31,14.261541
6944,IAU,2018-12-31,24.580000
9284,IGOV,2018-12-31,48.164463
11624,VEU,2018-12-31,41.742603
13964,VNQ,2018-12-31,66.764374
16304,VTI,2018-12-31,121.047112


In [58]:
adj_close.tail()

,Ticker,Date,Adj Close
16375,VTI,2019-04-12,141.773788
16376,VTI,2019-04-15,141.659348
16377,VTI,2019-04-16,141.697495
16378,VTI,2019-04-17,141.173019
16379,VTI,2019-04-18,141.382843


In [59]:
adj_close.tail()

,Ticker,Date,Adj Close
16375,VTI,2019-04-12,141.773788
16376,VTI,2019-04-15,141.659348
16377,VTI,2019-04-16,141.697495
16378,VTI,2019-04-17,141.173019
16379,VTI,2019-04-18,141.382843


In [60]:
# Grab the latest stock close price 

adj_close_latest = adj_close[adj_close['Date']==stocks_end]
adj_close_latest

,Ticker,Date,Adj Close
2339,AGG,2019-04-18,101.523415
4679,DBC,2019-04-18,15.954422
7019,IAU,2019-04-18,24.440001
9359,IGOV,2019-04-18,48.432762
11699,VEU,2019-04-18,47.527245
14039,VNQ,2019-04-18,77.088570
16379,VTI,2019-04-18,141.382843


## Dataframe Development

In [61]:
# Merge the portfolio dataframe with the adj close dataframe.

merged_portfolio = pd.merge(portfolio_df, adj_close_latest, on='Ticker')
merged_portfolio

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,Ticker #,Date,Adj Close
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,AGG 0,2019-04-18,101.523415
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,DBC 1,2019-04-18,15.954422
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,IAU 2,2019-04-18,24.440001
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,IGOV 3,2019-04-18,48.432762
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,VEU 4,2019-04-18,47.527245
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,VNQ 5,2019-04-18,77.088570
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,VTI 6,2019-04-18,141.382843
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,VTI 7,2019-04-18,141.382843


In [62]:
# The below creates a new column which is the ticker return; takes the latest adjusted close for each position
# and divides that by the initial share cost.

merged_portfolio['Ticker Return'] = merged_portfolio['Adj Close'] / merged_portfolio['Unit Cost'] - 1

merged_portfolio

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,Ticker #,Date,Adj Close,Ticker Return
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,AGG 0,2019-04-18,101.523415,0.060629
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,DBC 1,2019-04-18,15.954422,-0.391285
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,IAU 2,2019-04-18,24.440001,0.953637
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,IGOV 3,2019-04-18,48.432762,-0.045095
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,VEU 4,2019-04-18,47.527245,0.082132
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,VNQ 5,2019-04-18,77.088570,0.322728
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,VTI 6,2019-04-18,141.382843,0.602980
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,VTI 7,2019-04-18,141.382843,0.594483


In [63]:
# Above we reset the index to the newly merged dataframe.  This is because we have a flat dataframe for the sp500 returns
# and we merge the the new dataframe with the sp500 adjusted closes since the sp start on acquisition date and sp500 close date.

merged_portfolio_sp = pd.merge(merged_portfolio, sp_500_adj_close, left_on='Acquisition Date', right_on='Date', how='left')

merged_portfolio_sp

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,Ticker #,Date_x,Adj Close_x,Ticker Return,Date_y,Adj Close_y
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,AGG 0,2019-04-18,101.523415,0.060629,2014-04-21,1871.890015
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,DBC 1,2019-04-18,15.954422,-0.391285,2014-04-21,1871.890015
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,IAU 2,2019-04-18,24.440001,0.953637,2014-04-21,1871.890015
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,IGOV 3,2019-04-18,48.432762,-0.045095,2014-04-21,1871.890015
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,VEU 4,2019-04-18,47.527245,0.082132,2014-04-21,1871.890015
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,VNQ 5,2019-04-18,77.088570,0.322728,2014-04-21,1871.890015
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,VTI 6,2019-04-18,141.382843,0.602980,2014-04-21,1871.890015
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,VTI 7,2019-04-18,141.382843,0.594483,2014-05-05,1884.660034


In [64]:
# We will delete the additional date column which is created from this merge.
# We then rename columns to Latest Date and then reflect Ticker Adj Close and SP 500 Initial Close.

del merged_portfolio_sp['Date_y']

merged_portfolio_sp.rename(columns={'Date_x': 'Latest Date', 'Adj Close_x': 'Ticker Adj Close'
                                    , 'Adj Close_y': 'SP 500 Initial Close'}, inplace=True)

merged_portfolio_sp

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,Ticker #,Latest Date,Ticker Adj Close,Ticker Return,SP 500 Initial Close
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,AGG 0,2019-04-18,101.523415,0.060629,1871.890015
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,DBC 1,2019-04-18,15.954422,-0.391285,1871.890015
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,IAU 2,2019-04-18,24.440001,0.953637,1871.890015
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,IGOV 3,2019-04-18,48.432762,-0.045095,1871.890015
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,VEU 4,2019-04-18,47.527245,0.082132,1871.890015
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,VNQ 5,2019-04-18,77.088570,0.322728,1871.890015
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,VTI 6,2019-04-18,141.382843,0.602980,1871.890015
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,VTI 7,2019-04-18,141.382843,0.594483,1884.660034


In [65]:
# This new column is intended to figure out what SP 500 equivalent purchase would have been at purchase date of stock.
merged_portfolio_sp['Equiv SP Shares'] = merged_portfolio_sp['Cost Basis'] / merged_portfolio_sp['SP 500 Initial Close']
merged_portfolio_sp

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,Ticker #,Latest Date,Ticker Adj Close,Ticker Return,SP 500 Initial Close,Equiv SP Shares
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,AGG 0,2019-04-18,101.523415,0.060629,1871.890015,1.278387
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,DBC 1,2019-04-18,15.954422,-0.391285,1871.890015,0.308042
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,IAU 2,2019-04-18,24.440001,0.953637,1871.890015,0.334154
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,IGOV 3,2019-04-18,48.432762,-0.045095,1871.890015,0.216765
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,VEU 4,2019-04-18,47.527245,0.082132,1871.890015,2.463606
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,VNQ 5,2019-04-18,77.088570,0.322728,1871.890015,0.342477
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,VTI 6,2019-04-18,141.382843,0.602980,1871.890015,3.392507
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,VTI 7,2019-04-18,141.382843,0.594483,1884.660034,1.505545


In [66]:
sp_500_adj_close['Date']

0      2009-12-31
1      2010-01-04
2      2010-01-05
3      2010-01-06
4      2010-01-07
          ...    
2335   2019-04-12
2336   2019-04-15
2337   2019-04-16
2338   2019-04-17
2339   2019-04-18
Name: Date, Length: 2340, dtype: datetime64[ns]

In [67]:
merged_portfolio_sp['Latest Date']

0   2019-04-18
1   2019-04-18
2   2019-04-18
3   2019-04-18
4   2019-04-18
5   2019-04-18
6   2019-04-18
7   2019-04-18
Name: Latest Date, dtype: datetime64[ns]

In [68]:
# We are joining the developing dataframe with the sp500 closes again, this time with the latest close for SP.
merged_portfolio_sp_latest = pd.merge(merged_portfolio_sp, sp_500_adj_close, left_on='Latest Date', right_on='Date')

merged_portfolio_sp_latest

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,Ticker #,Latest Date,Ticker Adj Close,Ticker Return,SP 500 Initial Close,Equiv SP Shares,Date,Adj Close
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,AGG 0,2019-04-18,101.523415,0.060629,1871.890015,1.278387,2019-04-18,2905.030029
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,DBC 1,2019-04-18,15.954422,-0.391285,1871.890015,0.308042,2019-04-18,2905.030029
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,IAU 2,2019-04-18,24.440001,0.953637,1871.890015,0.334154,2019-04-18,2905.030029
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,IGOV 3,2019-04-18,48.432762,-0.045095,1871.890015,0.216765,2019-04-18,2905.030029
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,VEU 4,2019-04-18,47.527245,0.082132,1871.890015,2.463606,2019-04-18,2905.030029
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,VNQ 5,2019-04-18,77.088570,0.322728,1871.890015,0.342477,2019-04-18,2905.030029
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,VTI 6,2019-04-18,141.382843,0.602980,1871.890015,3.392507,2019-04-18,2905.030029
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,VTI 7,2019-04-18,141.382843,0.594483,1884.660034,1.505545,2019-04-18,2905.030029


In [69]:
# Once again need to delete the new Date column added as it's redundant to Latest Date.  
# Modify Adj Close from the sp dataframe to distinguish it by calling it the SP 500 Latest Close.

del merged_portfolio_sp_latest['Date']

merged_portfolio_sp_latest.rename(columns={'Adj Close': 'SP 500 Latest Close'}, inplace=True)

merged_portfolio_sp_latest

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,Ticker #,Latest Date,Ticker Adj Close,Ticker Return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,AGG 0,2019-04-18,101.523415,0.060629,1871.890015,1.278387,2905.030029
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,DBC 1,2019-04-18,15.954422,-0.391285,1871.890015,0.308042,2905.030029
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,IAU 2,2019-04-18,24.440001,0.953637,1871.890015,0.334154,2905.030029
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,IGOV 3,2019-04-18,48.432762,-0.045095,1871.890015,0.216765,2905.030029
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,VEU 4,2019-04-18,47.527245,0.082132,1871.890015,2.463606,2905.030029
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,VNQ 5,2019-04-18,77.088570,0.322728,1871.890015,0.342477,2905.030029
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,VTI 6,2019-04-18,141.382843,0.602980,1871.890015,3.392507,2905.030029
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,VTI 7,2019-04-18,141.382843,0.594483,1884.660034,1.505545,2905.030029


In [70]:
# Percent return of SP from acquisition date of position through latest trading day.
merged_portfolio_sp_latest['SP Return'] = merged_portfolio_sp_latest['SP 500 Latest Close'] / merged_portfolio_sp_latest['SP 500 Initial Close'] - 1

# This is a new column which takes the tickers return and subtracts the sp 500 equivalent range return.
merged_portfolio_sp_latest['Abs. Return Compare'] = merged_portfolio_sp_latest['Ticker Return'] - merged_portfolio_sp_latest['SP Return']

# This is a new column where we calculate the ticker's share value by multiplying the original quantity by the latest close.
merged_portfolio_sp_latest['Ticker Share Value'] = merged_portfolio_sp_latest['Quantity'] * merged_portfolio_sp_latest['Ticker Adj Close']

# We calculate the equivalent SP 500 Value if we take the original SP shares * the latest SP 500 share price.
merged_portfolio_sp_latest['SP 500 Value'] = merged_portfolio_sp_latest['Equiv SP Shares'] * merged_portfolio_sp_latest['SP 500 Latest Close']

# This is a new column where we take the current market value for the shares and subtract the SP 500 value.
merged_portfolio_sp_latest['Abs Value Compare'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['SP 500 Value']

# This column calculates profit / loss for stock position.
merged_portfolio_sp_latest['Stock Gain / (Loss)'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['Cost Basis']

# This column calculates profit / loss for SP 500.
merged_portfolio_sp_latest['SP 500 Gain / (Loss)'] = merged_portfolio_sp_latest['SP 500 Value'] - merged_portfolio_sp_latest['Cost Basis']

merged_portfolio_sp_latest

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,...,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss)
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,...,1871.890015,1.278387,2905.030029,0.551923,-0.491294,2538.085365,3713.752841,-1175.667476,145.085365,1320.752841
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,...,1871.890015,0.308042,2905.030029,0.551923,-0.943208,350.997284,894.870106,-543.872822,-225.622716,318.250106
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,...,1871.890015,0.334154,2905.030029,0.551923,0.401714,1222.000027,970.728125,251.271902,596.500027,345.228125
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,...,1871.890015,0.216765,2905.030029,0.551923,-0.597019,387.462097,629.708463,-242.246366,-18.297903,223.948463
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,...,1871.890015,2.463606,2905.030029,0.551923,-0.469791,4990.360680,7156.850231,-2166.489551,378.760680,2545.250231
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,...,1871.890015,0.342477,2905.030029,0.551923,-0.229196,847.974266,994.907092,-146.932826,206.894266,353.827092
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,...,1871.890015,3.392507,2905.030029,0.551923,0.051057,10179.564697,9855.334744,324.229953,3829.164697,3504.934744
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,...,1884.660034,1.505545,2905.030029,0.541408,0.053075,4524.250977,4373.652678,150.598298,1686.810977,1536.212678


In [71]:
# Merge the overall dataframe with the adj close start of year dataframe for YTD tracking.

merged_portfolio_sp_latest_YTD = pd.merge(merged_portfolio_sp_latest, adj_close_start, on='Ticker')
merged_portfolio_sp_latest_YTD

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,...,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Date,Adj Close
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,...,2905.030029,0.551923,-0.491294,2538.085365,3713.752841,-1175.667476,145.085365,1320.752841,2018-12-31,99.191551
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,...,2905.030029,0.551923,-0.943208,350.997284,894.870106,-543.872822,-225.622716,318.250106,2018-12-31,14.261541
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,...,2905.030029,0.551923,0.401714,1222.000027,970.728125,251.271902,596.500027,345.228125,2018-12-31,24.580000
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,...,2905.030029,0.551923,-0.597019,387.462097,629.708463,-242.246366,-18.297903,223.948463,2018-12-31,48.164463
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,...,2905.030029,0.551923,-0.469791,4990.360680,7156.850231,-2166.489551,378.760680,2545.250231,2018-12-31,41.742603
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,...,2905.030029,0.551923,-0.229196,847.974266,994.907092,-146.932826,206.894266,353.827092,2018-12-31,66.764374
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,...,2905.030029,0.551923,0.051057,10179.564697,9855.334744,324.229953,3829.164697,3504.934744,2018-12-31,121.047112
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,...,2905.030029,0.541408,0.053075,4524.250977,4373.652678,150.598298,1686.810977,1536.212678,2018-12-31,121.047112


In [72]:
# Deleting date again as it's an unnecessary column.  Explaining that new column is the Ticker Start of Year Close.

del merged_portfolio_sp_latest_YTD['Date']

merged_portfolio_sp_latest_YTD.rename(columns={'Adj Close': 'Ticker Start Year Close'}, inplace=True)

merged_portfolio_sp_latest_YTD.head()

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,...,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,...,1.278387,2905.030029,0.551923,-0.491294,2538.085365,3713.752841,-1175.667476,145.085365,1320.752841,99.191551
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,...,0.308042,2905.030029,0.551923,-0.943208,350.997284,894.870106,-543.872822,-225.622716,318.250106,14.261541
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,...,0.334154,2905.030029,0.551923,0.401714,1222.000027,970.728125,251.271902,596.500027,345.228125,24.580000
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,...,0.216765,2905.030029,0.551923,-0.597019,387.462097,629.708463,-242.246366,-18.297903,223.948463,48.164463
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,...,2.463606,2905.030029,0.551923,-0.469791,4990.360680,7156.850231,-2166.489551,378.760680,2545.250231,41.742603


In [73]:
# Join the SP 500 start of year with current dataframe.

merged_portfolio_sp_latest_YTD_sp = pd.merge(merged_portfolio_sp_latest_YTD, sp_500_adj_close_start
                                             , left_on='Start of Year', right_on='Date')

merged_portfolio_sp_latest_YTD_sp.head()

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,...,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,Date,Adj Close
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,...,0.551923,-0.491294,2538.085365,3713.752841,-1175.667476,145.085365,1320.752841,99.191551,2018-12-31,2506.850098
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,...,0.551923,-0.943208,350.997284,894.870106,-543.872822,-225.622716,318.250106,14.261541,2018-12-31,2506.850098
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,...,0.551923,0.401714,1222.000027,970.728125,251.271902,596.500027,345.228125,24.580000,2018-12-31,2506.850098
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,...,0.551923,-0.597019,387.462097,629.708463,-242.246366,-18.297903,223.948463,48.164463,2018-12-31,2506.850098
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,...,0.551923,-0.469791,4990.360680,7156.850231,-2166.489551,378.760680,2545.250231,41.742603,2018-12-31,2506.850098


In [74]:
# Deleting another unneeded Data column.

del merged_portfolio_sp_latest_YTD_sp['Date']

# Renaming so that it's clear this column is SP 500 start of year close.
merged_portfolio_sp_latest_YTD_sp.rename(columns={'Adj Close': 'SP Start Year Close'}, inplace=True)

# YTD return for portfolio position.
merged_portfolio_sp_latest_YTD_sp['Share YTD'] = merged_portfolio_sp_latest_YTD_sp['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp['Ticker Start Year Close'] - 1

# YTD return for SP to run compares.
merged_portfolio_sp_latest_YTD_sp['SP 500 YTD'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Latest Close'] / merged_portfolio_sp_latest_YTD_sp['SP Start Year Close'] - 1

merged_portfolio_sp_latest_YTD_sp

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,...,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,...,-0.491294,2538.085365,3713.752841,-1175.667476,145.085365,1320.752841,99.191551,2506.850098,0.023509,0.158837
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,...,-0.943208,350.997284,894.870106,-543.872822,-225.622716,318.250106,14.261541,2506.850098,0.118703,0.158837
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,...,0.401714,1222.000027,970.728125,251.271902,596.500027,345.228125,24.580000,2506.850098,-0.005696,0.158837
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,...,-0.597019,387.462097,629.708463,-242.246366,-18.297903,223.948463,48.164463,2506.850098,0.005570,0.158837
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,...,-0.469791,4990.360680,7156.850231,-2166.489551,378.760680,2545.250231,41.742603,2506.850098,0.138579,0.158837
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,...,-0.229196,847.974266,994.907092,-146.932826,206.894266,353.827092,66.764374,2506.850098,0.154636,0.158837
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,...,0.051057,10179.564697,9855.334744,324.229953,3829.164697,3504.934744,121.047112,2506.850098,0.167998,0.158837
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,...,0.053075,4524.250977,4373.652678,150.598298,1686.810977,1536.212678,121.047112,2506.850098,0.167998,0.158837


In [75]:
merged_portfolio_sp_latest_YTD_sp = merged_portfolio_sp_latest_YTD_sp.sort_values(by=['Ticker', 'Acquisition Date']
                                                                                  , ascending=[True, True])
merged_portfolio_sp_latest_YTD_sp

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,...,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,...,-0.491294,2538.085365,3713.752841,-1175.667476,145.085365,1320.752841,99.191551,2506.850098,0.023509,0.158837
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,...,-0.943208,350.997284,894.870106,-543.872822,-225.622716,318.250106,14.261541,2506.850098,0.118703,0.158837
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,...,0.401714,1222.000027,970.728125,251.271902,596.500027,345.228125,24.580000,2506.850098,-0.005696,0.158837
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,...,-0.597019,387.462097,629.708463,-242.246366,-18.297903,223.948463,48.164463,2506.850098,0.005570,0.158837
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,...,-0.469791,4990.360680,7156.850231,-2166.489551,378.760680,2545.250231,41.742603,2506.850098,0.138579,0.158837
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,...,-0.229196,847.974266,994.907092,-146.932826,206.894266,353.827092,66.764374,2506.850098,0.154636,0.158837
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,...,0.051057,10179.564697,9855.334744,324.229953,3829.164697,3504.934744,121.047112,2506.850098,0.167998,0.158837
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,...,0.053075,4524.250977,4373.652678,150.598298,1686.810977,1536.212678,121.047112,2506.850098,0.167998,0.158837


In [76]:
# Cumulative sum of original investment
merged_portfolio_sp_latest_YTD_sp['Cum Invst'] = merged_portfolio_sp_latest_YTD_sp['Cost Basis'].cumsum()

# Cumulative sum of Ticker Share Value (latest FMV based on initial quantity purchased).
merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] = merged_portfolio_sp_latest_YTD_sp['Ticker Share Value'].cumsum()

# Cumulative sum of SP Share Value (latest FMV driven off of initial SP equiv purchase).
merged_portfolio_sp_latest_YTD_sp['Cum SP Returns'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Value'].cumsum()

# Cumulative CoC multiple return for stock investments
merged_portfolio_sp_latest_YTD_sp['Cum Ticker ROI Mult'] = merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] / merged_portfolio_sp_latest_YTD_sp['Cum Invst']

# Cumulative Pct return for stock investments
merged_portfolio_sp_latest_YTD_sp['Cum Ticker ROI Pct'] = merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] / merged_portfolio_sp_latest_YTD_sp['Cum Invst'] - 1

# Cumulative Pct return for SP500
merged_portfolio_sp_latest_YTD_sp['Cum SP ROI Pct'] = merged_portfolio_sp_latest_YTD_sp['Cum SP Returns'] / merged_portfolio_sp_latest_YTD_sp['Cum Invst'] - 1

merged_portfolio_sp_latest_YTD_sp

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,...,Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult,Cum Ticker ROI Pct,Cum SP ROI Pct
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,...,99.191551,2506.850098,0.023509,0.158837,2393.00,2538.085365,3713.752841,1.060629,0.060629,0.551923
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,...,14.261541,2506.850098,0.118703,0.158837,2969.62,2889.082649,4608.622947,0.972880,-0.027120,0.551923
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,...,24.580000,2506.850098,-0.005696,0.158837,3595.12,4111.082676,5579.351071,1.143518,0.143518,0.551923
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,...,48.164463,2506.850098,0.005570,0.158837,4000.88,4498.544773,6209.059535,1.124389,0.124389,0.551923
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,...,41.742603,2506.850098,0.138579,0.158837,8612.48,9488.905453,13365.909765,1.101762,0.101762,0.551923
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,...,66.764374,2506.850098,0.154636,0.158837,9253.56,10336.879719,14360.816858,1.117071,0.117071,0.551923
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,...,121.047112,2506.850098,0.167998,0.158837,15603.96,20516.444416,24216.151601,1.314823,0.314823,0.551923
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,...,121.047112,2506.850098,0.167998,0.158837,18441.40,25040.695393,28589.804280,1.357852,0.357852,0.550306


In [77]:
# Only include positions where currently invested, if applicable, for subset of visualizations.

merged_portfolio_sp_latest_YTD_sp_chart = merged_portfolio_sp_latest_YTD_sp[merged_portfolio_sp_latest_YTD_sp['Unit Cost']>0]

merged_portfolio_sp_latest_YTD_sp_chart

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,...,Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult,Cum Ticker ROI Pct,Cum SP ROI Pct
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,...,99.191551,2506.850098,0.023509,0.158837,2393.00,2538.085365,3713.752841,1.060629,0.060629,0.551923
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,...,14.261541,2506.850098,0.118703,0.158837,2969.62,2889.082649,4608.622947,0.972880,-0.027120,0.551923
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,...,24.580000,2506.850098,-0.005696,0.158837,3595.12,4111.082676,5579.351071,1.143518,0.143518,0.551923
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,...,48.164463,2506.850098,0.005570,0.158837,4000.88,4498.544773,6209.059535,1.124389,0.124389,0.551923
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,...,41.742603,2506.850098,0.138579,0.158837,8612.48,9488.905453,13365.909765,1.101762,0.101762,0.551923
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,...,66.764374,2506.850098,0.154636,0.158837,9253.56,10336.879719,14360.816858,1.117071,0.117071,0.551923
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,...,121.047112,2506.850098,0.167998,0.158837,15603.96,20516.444416,24216.151601,1.314823,0.314823,0.551923
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,...,121.047112,2506.850098,0.167998,0.158837,18441.40,25040.695393,28589.804280,1.357852,0.357852,0.550306


In [78]:
# This dataframe will only look at the positions with an intended contribution, if applicable.

merged_portfolio_sp_latest_YTD_sp_contr = merged_portfolio_sp_latest_YTD_sp[merged_portfolio_sp_latest_YTD_sp['Target_Alloc']>0]

merged_portfolio_sp_latest_YTD_sp_contr

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,...,Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult,Cum Ticker ROI Pct,Cum SP ROI Pct
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,...,99.191551,2506.850098,0.023509,0.158837,2393.00,2538.085365,3713.752841,1.060629,0.060629,0.551923
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,...,14.261541,2506.850098,0.118703,0.158837,2969.62,2889.082649,4608.622947,0.972880,-0.027120,0.551923
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,...,24.580000,2506.850098,-0.005696,0.158837,3595.12,4111.082676,5579.351071,1.143518,0.143518,0.551923
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,...,48.164463,2506.850098,0.005570,0.158837,4000.88,4498.544773,6209.059535,1.124389,0.124389,0.551923
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,...,41.742603,2506.850098,0.138579,0.158837,8612.48,9488.905453,13365.909765,1.101762,0.101762,0.551923
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,...,66.764374,2506.850098,0.154636,0.158837,9253.56,10336.879719,14360.816858,1.117071,0.117071,0.551923
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,...,121.047112,2506.850098,0.167998,0.158837,15603.96,20516.444416,24216.151601,1.314823,0.314823,0.551923
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,...,121.047112,2506.850098,0.167998,0.158837,18441.40,25040.695393,28589.804280,1.357852,0.357852,0.550306


In [79]:
# Shorten dataframe to focus on columns that will be used to look at intended versus current allocations.

merged_portfolio_sp_latest_YTD_sp_contr_subset = merged_portfolio_sp_latest_YTD_sp_contr[['Ticker', 'Target_Alloc', 'Cost Basis', 'Ticker Share Value']]

merged_portfolio_sp_latest_YTD_sp_contr_subset

,Ticker,Target_Alloc,Cost Basis,Ticker Share Value
0,AGG,0.127,2393.00,2538.085365
1,DBC,0.033,576.62,350.997284
2,IAU,0.033,625.50,1222.000027
3,IGOV,0.023,405.76,387.462097
4,VEU,0.250,4611.60,4990.360680
5,VNQ,0.034,641.08,847.974266
6,VTI,0.500,6350.40,10179.564697
7,VTI,0.500,2837.44,4524.250977


In [80]:
# If you've bought multiple positions at different times, this pivot table will aggregate the sums for each position.

merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot = merged_portfolio_sp_latest_YTD_sp_contr_subset.pivot_table(
    index=['Ticker', 'Target_Alloc'], values='Ticker Share Value', aggfunc='sum')

merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot.reset_index(inplace=True)

merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot

,Ticker,Target_Alloc,Ticker Share Value
0,AGG,0.127,2538.085365
1,DBC,0.033,350.997284
2,IAU,0.033,1222.000027
3,IGOV,0.023,387.462097
4,VEU,0.250,4990.360680
5,VNQ,0.034,847.974266
6,VTI,0.500,14703.815674


In [81]:
merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot.columns

Index(['Ticker', 'Target_Alloc', 'Ticker Share Value'], dtype='object')

In [82]:
# These new columns calculate the actual allocation to compare to the target allocation.

merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot['Total'] = merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot.loc[:, 'Ticker Share Value'].cumsum()

merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot['Allocation'] = merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot['Ticker Share Value'] / merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot.iloc[-1, -1]

merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot

,Ticker,Target_Alloc,Ticker Share Value,Total,Allocation
0,AGG,0.127,2538.085365,2538.085365,0.101358
1,DBC,0.033,350.997284,2889.082649,0.014017
2,IAU,0.033,1222.000027,4111.082676,0.048801
3,IGOV,0.023,387.462097,4498.544773,0.015473
4,VEU,0.250,4990.360680,9488.905453,0.199290
5,VNQ,0.034,847.974266,10336.879719,0.033864
6,VTI,0.500,14703.815674,25040.695393,0.587197


In [83]:
merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot.sort_values(by='Target_Alloc', ascending=False, inplace=True)

merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot

,Ticker,Target_Alloc,Ticker Share Value,Total,Allocation
6,VTI,0.500,14703.815674,25040.695393,0.587197
4,VEU,0.250,4990.360680,9488.905453,0.199290
0,AGG,0.127,2538.085365,2538.085365,0.101358
5,VNQ,0.034,847.974266,10336.879719,0.033864
1,DBC,0.033,350.997284,2889.082649,0.014017
2,IAU,0.033,1222.000027,4111.082676,0.048801
3,IGOV,0.023,387.462097,4498.544773,0.015473


## Closing High Eval

In [84]:
# Referencing the adj_close dataframe from above

adj_close.head()

,Ticker,Date,Adj Close
0,AGG,2009-12-31,75.122749
1,AGG,2010-01-04,75.210129
2,AGG,2010-01-05,75.552269
3,AGG,2010-01-06,75.508575
4,AGG,2010-01-07,75.421249


In [85]:
portfolio_df.head()

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,Ticker #
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,AGG 0
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,DBC 1
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,IAU 2
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,IGOV 3
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,VEU 4


In [86]:
# Need to factor in that some positions were purchased much more recently than others.
# Join adj_close dataframe with portfolio in order to have acquisition date.

adj_close_acq_date = pd.merge(adj_close, portfolio_df, on='Ticker')

adj_close_acq_date.head()

,Ticker,Date,Adj Close,Acquisition Date,Position,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,Ticker #
0,AGG,2009-12-31,75.122749,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),25,95.72,2393.0,2018-12-31,0.127,NaN,0,AGG 0
1,AGG,2010-01-04,75.210129,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),25,95.72,2393.0,2018-12-31,0.127,NaN,0,AGG 0
2,AGG,2010-01-05,75.552269,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),25,95.72,2393.0,2018-12-31,0.127,NaN,0,AGG 0
3,AGG,2010-01-06,75.508575,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),25,95.72,2393.0,2018-12-31,0.127,NaN,0,AGG 0
4,AGG,2010-01-07,75.421249,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),25,95.72,2393.0,2018-12-31,0.127,NaN,0,AGG 0


In [87]:
# delete_columns = ['Quantity', 'Unit Cost', 'Cost Basis', 'Start of Year']

del adj_close_acq_date['Quantity']
del adj_close_acq_date['Unit Cost']
del adj_close_acq_date['Cost Basis']
del adj_close_acq_date['Start of Year']

adj_close_acq_date.sort_values(by=['Ticker', 'Acquisition Date', 'Date'], ascending=[True, True, True], inplace=True)

In [88]:
# Create a new column, Data Delta, to figure out the dates in time series post when the position was acquired.

adj_close_acq_date['Date Delta'] = adj_close_acq_date['Date'] - adj_close_acq_date['Acquisition Date']

adj_close_acq_date['Date Delta'] = adj_close_acq_date[['Date Delta']].apply(pd.to_numeric)  

adj_close_acq_date.head()

,Ticker,Date,Adj Close,Acquisition Date,Position,Target_Alloc,Comments,Counts,Ticker #,Date Delta
0,AGG,2009-12-31,75.122749,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),0.127,NaN,0,AGG 0,-135820800000000000
1,AGG,2010-01-04,75.210129,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),0.127,NaN,0,AGG 0,-135475200000000000
2,AGG,2010-01-05,75.552269,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),0.127,NaN,0,AGG 0,-135388800000000000
3,AGG,2010-01-06,75.508575,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),0.127,NaN,0,AGG 0,-135302400000000000
4,AGG,2010-01-07,75.421249,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),0.127,NaN,0,AGG 0,-135216000000000000


In [89]:
# Modified the dataframe being evaluated to look at highest close which occurred after Acquisition Date (aka, not prior to purchase).

adj_close_acq_date_modified = adj_close_acq_date[adj_close_acq_date['Date Delta']>=0]

adj_close_acq_date_modified.head()

,Ticker,Date,Adj Close,Acquisition Date,Position,Target_Alloc,Comments,Counts,Ticker #,Date Delta
1081,AGG,2014-04-21,89.461205,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),0.127,NaN,0,AGG 0,0
1082,AGG,2014-04-22,89.444656,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),0.127,NaN,0,AGG 0,86400000000000
1083,AGG,2014-04-23,89.535622,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),0.127,NaN,0,AGG 0,172800000000000
1084,AGG,2014-04-24,89.618324,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),0.127,NaN,0,AGG 0,259200000000000
1085,AGG,2014-04-25,89.634865,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),0.127,NaN,0,AGG 0,345600000000000


In [90]:
# This pivot table will index on the Ticker and Acquisition Date, and find the max adjusted close.

adj_close_pivot = adj_close_acq_date_modified.pivot_table(index=['Ticker', 'Acquisition Date'], values='Adj Close', aggfunc=np.max)

adj_close_pivot.reset_index(inplace=True)

adj_close_pivot[0:10]

,Ticker,Acquisition Date,Adj Close
0,AGG,2014-04-21,102.112717
1,DBC,2014-04-21,26.155830
2,IAU,2014-04-21,26.360001
3,IGOV,2014-04-21,51.578995
4,VEU,2014-04-21,52.347305
5,VNQ,2014-04-21,79.396973
6,VTI,2014-04-21,141.990143
7,VTI,2014-05-05,141.990143


In [91]:
# Merge the adj close pivot table with the adj_close table in order to grab the date of the Adj Close High (good to know).

adj_close_pivot_merged = pd.merge(adj_close_pivot, adj_close
                                             , on=['Ticker', 'Adj Close'])

adj_close_pivot_merged.head()

,Ticker,Acquisition Date,Adj Close,Date
0,AGG,2014-04-21,102.112717,2019-03-28
1,DBC,2014-04-21,26.155830,2013-04-09
2,DBC,2014-04-21,26.155830,2014-06-20
3,IAU,2014-04-21,26.360001,2010-10-08
4,IAU,2014-04-21,26.360001,2016-07-08


In [92]:
# Duplicates were created where the stock had the same high on multiple dates.
# Sorted by latest date and then dropped duplicates, which drops the earlier high from consideration.

adj_close_pivot_merged.sort_values(by=['Ticker', 'Acquisition Date', 'Date'], ascending=[True, True, False], inplace=True)

adj_close_pivot_merged.drop_duplicates(['Ticker', 'Acquisition Date', 'Adj Close'], inplace=True)

adj_close_pivot_merged.head()

,Ticker,Acquisition Date,Adj Close,Date
0,AGG,2014-04-21,102.112717,2019-03-28
2,DBC,2014-04-21,26.155830,2014-06-20
4,IAU,2014-04-21,26.360001,2016-07-08
5,IGOV,2014-04-21,51.578995,2018-03-26
6,VEU,2014-04-21,52.347305,2018-01-26


In [93]:
# Merge the Adj Close pivot table with the master dataframe to have the closing high since you have owned the stock.

merged_portfolio_sp_latest_YTD_sp_closing_high = pd.merge(merged_portfolio_sp_latest_YTD_sp, adj_close_pivot_merged
                                             , on=['Ticker', 'Acquisition Date'])

# Renaming so that it's clear that the new columns are two year closing high and two year closing high date.
merged_portfolio_sp_latest_YTD_sp_closing_high.rename(columns={'Adj Close': 'Closing High Adj Close', 'Date': 'Closing High Adj Close Date'}, inplace=True)

merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp_closing_high['Closing High Adj Close'] - 1 

merged_portfolio_sp_latest_YTD_sp_closing_high.head()

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,...,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult,Cum Ticker ROI Pct,Cum SP ROI Pct,Closing High Adj Close,Closing High Adj Close Date,Pct off High
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,NaN,0,...,0.158837,2393.00,2538.085365,3713.752841,1.060629,0.060629,0.551923,102.112717,2019-03-28,-0.005771
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,NaN,1,...,0.158837,2969.62,2889.082649,4608.622947,0.972880,-0.027120,0.551923,26.155830,2014-06-20,-0.390024
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,NaN,2,...,0.158837,3595.12,4111.082676,5579.351071,1.143518,0.143518,0.551923,26.360001,2016-07-08,-0.072838
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,...,0.158837,4000.88,4498.544773,6209.059535,1.124389,0.124389,0.551923,51.578995,2018-03-26,-0.060998
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,...,0.158837,8612.48,9488.905453,13365.909765,1.101762,0.101762,0.551923,52.347305,2018-01-26,-0.092078


In [94]:
merged_portfolio_sp_latest_YTD_sp_closing_high.tail()

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,...,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult,Cum Ticker ROI Pct,Cum SP ROI Pct,Closing High Adj Close,Closing High Adj Close Date,Pct off High
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,NaN,3,...,0.158837,4000.88,4498.544773,6209.059535,1.124389,0.124389,0.551923,51.578995,2018-03-26,-0.060998
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,NaN,4,...,0.158837,8612.48,9488.905453,13365.909765,1.101762,0.101762,0.551923,52.347305,2018-01-26,-0.092078
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,NaN,5,...,0.158837,9253.56,10336.879719,14360.816858,1.117071,0.117071,0.551923,79.396973,2019-04-12,-0.029074
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,NaN,6,...,0.158837,15603.96,20516.444416,24216.151601,1.314823,0.314823,0.551923,141.990143,2018-09-20,-0.004277
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,NaN,7,...,0.158837,18441.40,25040.695393,28589.804280,1.357852,0.357852,0.550306,141.990143,2018-09-20,-0.004277


## Performance Visualizations

### Allocation relative to Target

In [95]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot['Target_Alloc'],
    name = 'Target Allocation')

trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot['Allocation'],
    name = 'Current Allocation')
    
data = [trace1, trace2]

layout = go.Layout(title = 'Target Allocation versus Current Allocation'
    , barmode = 'group'
    , yaxis=dict(title='Allocations', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=0.8,y=1.2)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

### YTD Charts.

In [96]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_chart['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_chart['Share YTD'],
    name = 'Ticker YTD')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_chart['Ticker'],
    y = merged_portfolio_sp_latest_YTD_sp_chart['SP 500 YTD'],
    name = 'SP500 YTD')
    
data = [trace1, trace2]

layout = go.Layout(title = 'Total Return vs S&P 500, YTD'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=0.8,y=1.2)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Total Return.

In [97]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_chart['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_chart['Ticker Return'],
    name = 'Ticker Total Return')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_chart['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_chart['SP Return'],
    name = 'SP500 Total Return')
    
data = [trace1, trace2]

layout = go.Layout(title = 'Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1.2)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Total Cumulative Investments Over Time

In [98]:
trace1 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_chart['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_chart['Cum Invst'],
    mode = 'lines+markers',
    name = 'Cum Invst')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_chart['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_chart['Cum Ticker Returns'],
    mode = 'lines+markers',
    name = 'Cum Ticker Returns')

trace3 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_chart['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_chart['Cum SP Returns'],
    mode = 'lines+markers',
    name = 'Cum SP500 Returns')

data = [trace1, trace2, trace3]

layout = go.Layout(title = 'Total Investment Comparisons by Ticker'
    , barmode = 'group'
    , yaxis=dict(title='Returns')
    , xaxis=dict(title='Ticker')
    , legend=dict(x=1,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [99]:
# Graphing the total return percents.

trace1 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_chart['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_chart['Cum Ticker ROI Pct'],
    mode = 'lines+markers',
    name = 'Cum Ticker ROI Pct')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_chart['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_chart['Cum SP ROI Pct'],
    mode = 'lines+markers',
    name = 'Cum SP ROI Pct')

data = [trace1, trace2]

layout = go.Layout(title = 'Pct Comparisons by Ticker'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=1,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

## Closing High Visualizations

In [100]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker #'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'][0:10],
    name = 'Pct off High')

data = [trace1]

layout = go.Layout(title = 'Adj Close % off of High Since Purchased'
    , barmode = 'group'
    , yaxis=dict(title='% Below Adj Close High Since Purchased', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

## Dividends

In [101]:
# https://www.streetinsider.com/dividend_history.php?q=bdx

# https://www.nasdaq.com/symbol/rgld/dividend-history

# https://seekingalpha.com/symbol/EXG/dividends/history

# SP500 dividends:  https://ycharts.com/indicators/sp_500_dividends_per_share

In [102]:
# Import the Historical Dividends worksheet with ex-dividend dates and dividend amounts:

dividend_df = pd.read_excel('Historical Dividends.xlsx', sheet_name='Historical Dividends'
                            , parse_dates=['Ex-Div. Date', 'Decl. Date', 'Rec. Date', 'Pay. Date'])

dividend_df.head(10)

,Ticker,Ex-Div. Date,Amount,Type,Yield,Change,Decl. Date,Rec. Date,Pay. Date,Comments
0,AGG,2019-04-01,0.262,NaN,NaN,NaN,2018-12-13 00:00:00,2019-04-02 00:00:00,2019-04-05 00:00:00,NaN
1,AGG,2019-03-01,0.267,NaN,NaN,NaN,2018-12-13 00:00:00,2019-03-04 00:00:00,2019-03-07 00:00:00,NaN
2,AGG,2019-02-01,0.268,NaN,NaN,NaN,2018-12-13 00:00:00,2019-02-04 00:00:00,2019-02-07 00:00:00,NaN
3,AGG,2018-12-18,0.210,NaN,NaN,NaN,2018-01-03 00:00:00,2018-12-19 00:00:00,2018-12-24 00:00:00,NaN
4,AGG,2018-11-01,0.256,NaN,NaN,NaN,2018-01-03 00:00:00,2018-11-02 00:00:00,2018-11-07 00:00:00,NaN
5,AGG,2018-10-01,0.252,NaN,NaN,NaN,2018-01-03 00:00:00,2018-10-02 00:00:00,2018-10-05 00:00:00,NaN
6,AGG,2018-09-04,0.232,NaN,NaN,NaN,2018-01-03 00:00:00,2018-09-05 00:00:00,2018-09-10 00:00:00,NaN
7,AGG,2018-08-01,0.251,NaN,NaN,NaN,2018-01-03 00:00:00,2018-08-02 00:00:00,2018-08-07 00:00:00,NaN
8,AGG,2018-07-02,0.229,NaN,NaN,NaN,2018-01-03 00:00:00,2018-07-03 00:00:00,2018-07-09 00:00:00,NaN
9,AGG,2018-06-01,0.249,NaN,NaN,NaN,2018-01-03 00:00:00,2018-06-04 00:00:00,2018-06-07 00:00:00,NaN


In [103]:
# Filter a separate dividend df to only have the SP500 dividends.

dividend_df_sp500 = dividend_df[dividend_df['Ticker']=='^GSPC']

dividend_df_sp500.head()

,Ticker,Ex-Div. Date,Amount,Type,Yield,Change,Decl. Date,Rec. Date,Pay. Date,Comments
213,^GSPC,2018-12-31,14.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,^GSPC,2018-09-30,13.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,^GSPC,2018-06-30,13.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,^GSPC,2018-03-31,12.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,^GSPC,2017-12-31,12.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
# subset the dividend df for the needed columns.

dividend_df_subset = dividend_df[['Ticker', 'Ex-Div. Date', 'Amount']]

dividend_df_subset.head()

,Ticker,Ex-Div. Date,Amount
0,AGG,2019-04-01,0.262
1,AGG,2019-03-01,0.267
2,AGG,2019-02-01,0.268
3,AGG,2018-12-18,0.210
4,AGG,2018-11-01,0.256


In [105]:
# subset the portfolio df for needed columns.

portfolio_df_subset = merged_portfolio_sp_latest_YTD_sp_closing_high[['Ticker #', 'Acquisition Date', 'Ticker', 'Quantity'
                                                                      , 'Latest Date', 'Equiv SP Shares']]

portfolio_df_subset.tail()

,Ticker #,Acquisition Date,Ticker,Quantity,Latest Date,Equiv SP Shares
3,IGOV 3,2014-04-21,IGOV,8,2019-04-18,0.216765
4,VEU 4,2014-04-21,VEU,105,2019-04-18,2.463606
5,VNQ 5,2014-04-21,VNQ,11,2019-04-18,0.342477
6,VTI 6,2014-04-21,VTI,72,2019-04-18,3.392507
7,VTI 7,2014-05-05,VTI,32,2019-04-18,1.505545


In [106]:
# Merge the two subsets with the data needed to pull in the dividends received during ownership.

merged_subsets = pd.merge(portfolio_df_subset, dividend_df_subset, on='Ticker', how='left')

merged_subsets.tail()

,Ticker #,Acquisition Date,Ticker,Quantity,Latest Date,Equiv SP Shares,Ex-Div. Date,Amount
233,VTI 7,2014-05-05,VTI,32,2019-04-18,1.505545,2014-06-24,0.420
234,VTI 7,2014-05-05,VTI,32,2019-04-18,1.505545,2014-03-25,0.423
235,VTI 7,2014-05-05,VTI,32,2019-04-18,1.505545,2013-12-20,0.494
236,VTI 7,2014-05-05,VTI,32,2019-04-18,1.505545,2013-09-23,0.429
237,VTI 7,2014-05-05,VTI,32,2019-04-18,1.505545,2013-06-24,0.386


In [107]:
# This function determines if you owned the stock and were eligible to be paid the dividend.

def dividend_post_acquisition(df):
    if df['Ex-Div. Date'] > df['Acquisition Date'] and df['Ex-Div. Date'] <= stocks_end:
        val = 1
    elif df['Ex-Div. Date'] <= df['Acquisition Date']:
        val = 0
    else:
        val = 0
    return val

In [108]:
# Acq indicator of 1 means that you were eligible and received the dividend.

merged_subsets['Acq Indicator'] = merged_subsets.apply(dividend_post_acquisition, axis=1)

merged_subsets['Amount'].fillna(0, inplace=True)

merged_subsets.head()

,Ticker #,Acquisition Date,Ticker,Quantity,Latest Date,Equiv SP Shares,Ex-Div. Date,Amount,Acq Indicator
0,AGG 0,2014-04-21,AGG,25,2019-04-18,1.278387,2019-04-01,0.262,1
1,AGG 0,2014-04-21,AGG,25,2019-04-18,1.278387,2019-03-01,0.267,1
2,AGG 0,2014-04-21,AGG,25,2019-04-18,1.278387,2019-02-01,0.268,1
3,AGG 0,2014-04-21,AGG,25,2019-04-18,1.278387,2018-12-18,0.210,1
4,AGG 0,2014-04-21,AGG,25,2019-04-18,1.278387,2018-11-01,0.256,1


In [109]:
# filter the df on the dividends where you were eligible, calculate dividend based on share ownership.

merged_subsets_eligible = merged_subsets[merged_subsets['Acq Indicator']==1]

merged_subsets_eligible.loc[:, 'Dividend Amt'] = merged_subsets.loc[:, 'Amount'] * merged_subsets.loc[:, 'Quantity']

# merged_subsets_eligible['Date'] = pd.to_datetime(merged_subsets['Date'])

merged_subsets_eligible.loc[:, 'Year'] = merged_subsets.loc[:, 'Ex-Div. Date'].dt.year

merged_subsets_eligible.head()

/home/rootdaenib/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Ticker #,Acquisition Date,Ticker,Quantity,Latest Date,Equiv SP Shares,Ex-Div. Date,Amount,Acq Indicator,Dividend Amt,Year
0,AGG 0,2014-04-21,AGG,25,2019-04-18,1.278387,2019-04-01,0.262,1,6.550,2019.0
1,AGG 0,2014-04-21,AGG,25,2019-04-18,1.278387,2019-03-01,0.267,1,6.675,2019.0
2,AGG 0,2014-04-21,AGG,25,2019-04-18,1.278387,2019-02-01,0.268,1,6.700,2019.0
3,AGG 0,2014-04-21,AGG,25,2019-04-18,1.278387,2018-12-18,0.210,1,5.250,2018.0
4,AGG 0,2014-04-21,AGG,25,2019-04-18,1.278387,2018-11-01,0.256,1,6.400,2018.0


In [110]:
# This df is more useful as it breaks out dividends by year paid as well as total received during ownership.

merged_subsets_eligible_by_year = merged_subsets_eligible.pivot_table(index=['Ticker #', 'Ticker', 'Acquisition Date'],
                                                                          columns='Year', values='Dividend Amt', aggfunc=sum)

merged_subsets_eligible_by_year.reset_index(inplace=True)

merged_subsets_eligible_by_year.fillna(0, inplace=True)

merged_subsets_eligible_by_year['Dividend Amt'] = merged_subsets_eligible_by_year.iloc[:, 3:].sum(axis=1)

# Check out subtotals here:  http://pbpython.com/excel-pandas-comp.html

merged_subsets_eligible_by_year

Year,Ticker #,Ticker,Acquisition Date,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,Dividend Amt
0,AGG 0,AGG,2014-04-21,48.400000,62.150000,48.17500,57.625000,65.97500,19.925,302.250000
1,DBC 1,DBC,2014-04-21,0.000000,0.000000,0.00000,0.000000,4.15800,0.000,4.158000
2,IGOV 3,IGOV,2014-04-21,7.103696,0.839192,4.99232,1.326384,1.20788,0.000,15.469472
3,VEU 4,VEU,2014-04-21,131.985000,134.610000,137.34000,152.775000,156.34500,18.795,731.850000
4,VNQ 5,VNQ,2014-04-21,27.126000,34.364000,43.73600,38.654000,38.84100,6.776,189.497000
5,VTI 6,VTI,2014-04-21,104.112000,148.824000,159.48000,168.696000,187.48800,55.584,824.184000
6,VTI 7,VTI,2014-05-05,46.272000,66.144000,70.88000,74.976000,83.32800,24.704,366.304000


In [111]:
# Drop duplicates in order to get the latest dividend for each stock.

merged_subsets_eligible_latest = merged_subsets_eligible

merged_subsets_eligible_latest.sort_values(by=['Ticker #', 'Ex-Div. Date']
                                                                     , ascending=[True, False], inplace=True)

merged_subsets_eligible_latest.drop_duplicates(subset=['Ticker #'], keep='first', inplace=True)

merged_subsets_eligible_latest

/home/rootdaenib/.local/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Ticker #,Acquisition Date,Ticker,Quantity,Latest Date,Equiv SP Shares,Ex-Div. Date,Amount,Acq Indicator,Dividend Amt,Year
0,AGG 0,2014-04-21,AGG,25,2019-04-18,1.278387,2019-04-01,0.262000,1,6.55000,2019.0
64,DBC 1,2014-04-21,DBC,22,2019-04-18,0.308042,2018-12-24,0.189000,1,4.15800,2018.0
66,IGOV 3,2014-04-21,IGOV,8,2019-04-18,0.216765,2018-12-18,0.150985,1,1.20788,2018.0
142,VEU 4,2014-04-21,VEU,105,2019-04-18,2.463606,2019-03-25,0.179000,1,18.79500,2019.0
166,VNQ 5,2014-04-21,VNQ,11,2019-04-18,0.342477,2019-03-28,0.616000,1,6.77600,2019.0
190,VTI 6,2014-04-21,VTI,72,2019-04-18,3.392507,2019-03-25,0.772000,1,55.58400,2019.0
214,VTI 7,2014-05-05,VTI,32,2019-04-18,1.505545,2019-03-25,0.772000,1,24.70400,2019.0


In [112]:
merged_subsets_eligible_latest_subset = merged_subsets_eligible_latest[['Ticker #', 'Ex-Div. Date', 'Amount']]

merged_subsets_eligible_latest_subset.rename(columns={'Amount': 'Latest Div Amt'}, inplace=True)

merged_subsets_eligible_latest_subset

/home/rootdaenib/.local/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Ticker #,Ex-Div. Date,Latest Div Amt
0,AGG 0,2019-04-01,0.262000
64,DBC 1,2018-12-24,0.189000
66,IGOV 3,2018-12-18,0.150985
142,VEU 4,2019-03-25,0.179000
166,VNQ 5,2019-03-28,0.616000
190,VTI 6,2019-03-25,0.772000
214,VTI 7,2019-03-25,0.772000


In [113]:
# subset the df with the needed columns that includes total dividends received for each position.

merged_subsets_eligible_total = merged_subsets_eligible.pivot_table(index=['Ticker #', 'Ticker', 'Acquisition Date', 'Quantity'
                                                                           , 'Latest Date', 'Equiv SP Shares']
                                                                          , values='Dividend Amt', aggfunc=sum)

merged_subsets_eligible_total.reset_index(inplace=True)

merged_subsets_eligible_total

,Ticker #,Ticker,Acquisition Date,Quantity,Latest Date,Equiv SP Shares,Dividend Amt
0,AGG 0,AGG,2014-04-21,25,2019-04-18,1.278387,6.55000
1,DBC 1,DBC,2014-04-21,22,2019-04-18,0.308042,4.15800
2,IGOV 3,IGOV,2014-04-21,8,2019-04-18,0.216765,1.20788
3,VEU 4,VEU,2014-04-21,105,2019-04-18,2.463606,18.79500
4,VNQ 5,VNQ,2014-04-21,11,2019-04-18,0.342477,6.77600
5,VTI 6,VTI,2014-04-21,72,2019-04-18,3.392507,55.58400
6,VTI 7,VTI,2014-05-05,32,2019-04-18,1.505545,24.70400


In [114]:
dividend_df_sp500.head()

,Ticker,Ex-Div. Date,Amount,Type,Yield,Change,Decl. Date,Rec. Date,Pay. Date,Comments
213,^GSPC,2018-12-31,14.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,^GSPC,2018-09-30,13.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,^GSPC,2018-06-30,13.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,^GSPC,2018-03-31,12.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,^GSPC,2017-12-31,12.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
# This df adds all of the columns with the SP500 dividends paid during the range that each stock position was held.
# For comparative holding period purposes.

dividend_df_sp500_2 = dividend_df_sp500

agg0_start_date = datetime.datetime(2014, 4, 21)
dbc1_start_date = datetime.datetime(2014, 4, 21)
igov3_start_date = datetime.datetime(2014, 4, 21)
veu4_start_date = datetime.datetime(2014, 4, 21)
vnq5_start_date = datetime.datetime(2014, 4, 21)
vti6_start_date = datetime.datetime(2014, 4, 21)
vti7_start_date = datetime.datetime(2014, 5, 5)

dividend_df_sp500_2.loc[:, 'agg0_sum'] = dividend_df_sp500_2[(dividend_df_sp500_2['Ex-Div. Date'] > agg0_start_date) & (dividend_df_sp500_2['Ex-Div. Date'] <= stocks_end)].sum()['Amount']
dividend_df_sp500_2.loc[:, 'dbc1_sum'] = dividend_df_sp500_2[(dividend_df_sp500_2['Ex-Div. Date'] > dbc1_start_date) & (dividend_df_sp500_2['Ex-Div. Date'] <= stocks_end)].sum()['Amount']
dividend_df_sp500_2.loc[:, 'igov3_sum'] = dividend_df_sp500_2[(dividend_df_sp500_2['Ex-Div. Date'] > igov3_start_date) & (dividend_df_sp500_2['Ex-Div. Date'] <= stocks_end)].sum()['Amount']
dividend_df_sp500_2.loc[:, 'veu4_sum'] = dividend_df_sp500_2[(dividend_df_sp500_2['Ex-Div. Date'] > veu4_start_date) & (dividend_df_sp500_2['Ex-Div. Date'] <= stocks_end)].sum()['Amount']
dividend_df_sp500_2.loc[:, 'vnq5_sum'] = dividend_df_sp500_2[(dividend_df_sp500_2['Ex-Div. Date'] > vnq5_start_date) & (dividend_df_sp500_2['Ex-Div. Date'] <= stocks_end)].sum()['Amount']
dividend_df_sp500_2.loc[:, 'vti6_sum'] = dividend_df_sp500_2[(dividend_df_sp500_2['Ex-Div. Date'] > vti6_start_date) & (dividend_df_sp500_2['Ex-Div. Date'] <= stocks_end)].sum()['Amount']
dividend_df_sp500_2.loc[:, 'vti7_sum'] = dividend_df_sp500_2[(dividend_df_sp500_2['Ex-Div. Date'] > vti7_start_date) & (dividend_df_sp500_2['Ex-Div. Date'] <= stocks_end)].sum()['Amount']

dividend_df_sp500_2.head()

<ipython-input-115-91488386e903>:14: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

/home/rootdaenib/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-115-91488386e903>:15: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

<ipython-input-115-91488386e903>:16: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_o

,Ticker,Ex-Div. Date,Amount,Type,Yield,Change,Decl. Date,Rec. Date,Pay. Date,Comments,agg0_sum,dbc1_sum,igov3_sum,veu4_sum,vnq5_sum,vti6_sum,vti7_sum
213,^GSPC,2018-12-31,14.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,221.999,221.999,221.999,221.999,221.999,221.999,221.999
214,^GSPC,2018-09-30,13.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,221.999,221.999,221.999,221.999,221.999,221.999,221.999
215,^GSPC,2018-06-30,13.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,221.999,221.999,221.999,221.999,221.999,221.999,221.999
216,^GSPC,2018-03-31,12.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,221.999,221.999,221.999,221.999,221.999,221.999,221.999
217,^GSPC,2017-12-31,12.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,221.999,221.999,221.999,221.999,221.999,221.999,221.999


In [116]:
# Subset the above df and re-label the SP500 total dividend amount column.

dividend_df_sp500_2 = dividend_df_sp500_2.iloc[0, 10:]

dividend_df_sp500_2 = dividend_df_sp500_2.to_frame().reset_index()

dividend_df_sp500_2.rename(columns={213: 'SP500 Div Amt'}, inplace=True)

dividend_df_sp500_2

,index,SP500 Div Amt
0,agg0_sum,221.999
1,dbc1_sum,221.999
2,igov3_sum,221.999
3,veu4_sum,221.999
4,vnq5_sum,221.999
5,vti6_sum,221.999
6,vti7_sum,221.999


In [117]:
# This function is used to assign the Ticker # column so that this df can be joined with the master df.

def ticker_assigner(df):
    if df['index'] == 'agg0_sum':
        val = 'AGG 0'
    elif df['index'] == 'dbc1_sum':
        val = 'DBC 1'
    elif df['index'] == 'igov3_sum':
        val = 'IGOV 3'
    elif df['index'] == 'veu4_sum':
        val = 'VEU 4'
    elif df['index'] == 'vnq5_sum':
        val = 'VNQ 5'
    elif df['index'] == 'vti6_sum':
        val = 'VTI 6'
    elif df['index'] == 'vti7_sum':
        val = 'VTI 7'
    else:
        val = 0
    return val

In [118]:
dividend_df_sp500_2['Ticker #'] = dividend_df_sp500_2.apply(ticker_assigner, axis=1)

# merged_subsets['Amount'].fillna(0, inplace=True)

dividend_df_sp500_2

,index,SP500 Div Amt,Ticker #
0,agg0_sum,221.999,AGG 0
1,dbc1_sum,221.999,DBC 1
2,igov3_sum,221.999,IGOV 3
3,veu4_sum,221.999,VEU 4
4,vnq5_sum,221.999,VNQ 5
5,vti6_sum,221.999,VTI 6
6,vti7_sum,221.999,VTI 7


In [119]:
# Join the portfolio dividend df with the SP 500 df by Ticker # for comparative purposes.

dividend_sp_merged_final = pd.merge(merged_subsets_eligible_by_year, dividend_df_sp500_2, on='Ticker #', how='left')

del dividend_sp_merged_final['index']

# Rolling this back as include it later on when merge with master dataframe.
# dividend_sp_merged_final.loc[:, 'SP500 Tot Div Amt'] = dividend_sp_merged_final.loc[:, 'Equiv SP Shares'] * dividend_sp_merged_final.loc[:, 'SP500 Div Amt']

dividend_sp_merged_final

,Ticker #,Ticker,Acquisition Date,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,Dividend Amt,SP500 Div Amt
0,AGG 0,AGG,2014-04-21,48.400000,62.150000,48.17500,57.625000,65.97500,19.925,302.250000,221.999
1,DBC 1,DBC,2014-04-21,0.000000,0.000000,0.00000,0.000000,4.15800,0.000,4.158000,221.999
2,IGOV 3,IGOV,2014-04-21,7.103696,0.839192,4.99232,1.326384,1.20788,0.000,15.469472,221.999
3,VEU 4,VEU,2014-04-21,131.985000,134.610000,137.34000,152.775000,156.34500,18.795,731.850000,221.999
4,VNQ 5,VNQ,2014-04-21,27.126000,34.364000,43.73600,38.654000,38.84100,6.776,189.497000,221.999
5,VTI 6,VTI,2014-04-21,104.112000,148.824000,159.48000,168.696000,187.48800,55.584,824.184000,221.999
6,VTI 7,VTI,2014-05-05,46.272000,66.144000,70.88000,74.976000,83.32800,24.704,366.304000,221.999


In [120]:
dividend_sp_merged_final_subset = dividend_sp_merged_final[['Ticker #', 2014.0, 2015.0, 2016.0, 2017.0, 2018.0
                                                            , 2019.0, 'Dividend Amt', 'SP500 Div Amt']]

# Merging with the latest dividend dataframe in order to calculate current yield and holdings' yields.

dividend_sp_merged_final_subset = pd.merge(dividend_sp_merged_final_subset, merged_subsets_eligible_latest_subset
                                           , on='Ticker #') 

# Annualize the dividend.
dividend_sp_merged_final_subset['Latest Div Amt'] = dividend_sp_merged_final_subset['Latest Div Amt'] * 4.0

dividend_sp_merged_final_subset

,Ticker #,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,Dividend Amt,SP500 Div Amt,Ex-Div. Date,Latest Div Amt
0,AGG 0,48.400000,62.150000,48.17500,57.625000,65.97500,19.925,302.250000,221.999,2019-04-01,1.04800
1,DBC 1,0.000000,0.000000,0.00000,0.000000,4.15800,0.000,4.158000,221.999,2018-12-24,0.75600
2,IGOV 3,7.103696,0.839192,4.99232,1.326384,1.20788,0.000,15.469472,221.999,2018-12-18,0.60394
3,VEU 4,131.985000,134.610000,137.34000,152.775000,156.34500,18.795,731.850000,221.999,2019-03-25,0.71600
4,VNQ 5,27.126000,34.364000,43.73600,38.654000,38.84100,6.776,189.497000,221.999,2019-03-28,2.46400
5,VTI 6,104.112000,148.824000,159.48000,168.696000,187.48800,55.584,824.184000,221.999,2019-03-25,3.08800
6,VTI 7,46.272000,66.144000,70.88000,74.976000,83.32800,24.704,366.304000,221.999,2019-03-25,3.08800


In [121]:
# Master dataframe to also include all of the dividend info.

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs = pd.merge(merged_portfolio_sp_latest_YTD_sp_closing_high
                                                                    , dividend_sp_merged_final_subset, on='Ticker #', how='left')

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.fillna(0, inplace=True)

# This column will calculate the total SP 500 dividend amount.
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'SP500 Tot Div'] = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['SP500 Div Amt'] * merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Equiv SP Shares']

# Total shareholder return (TSR) value for position.
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'Total Stock Gain / (Loss)'] = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Stock Gain / (Loss)'] + merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Dividend Amt']

# Total shareholder return value for SP500 equivalent.
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'Total SP500 Gain / (Loss)'] = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['SP 500 Gain / (Loss)'] + merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['SP500 Tot Div']

# TSR % for position.
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'Total Stock Return'] = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Total Stock Gain / (Loss)'] / merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Cost Basis']

# TSR % for SP500.
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'Total SP500 Return'] = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Total SP500 Gain / (Loss)'] / merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Cost Basis']

# Total dividend yield from position.
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'Share Yield'] = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Dividend Amt'] / merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Cost Basis']

# Aggregate SP returns across all positions.
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'Cum Total SP Returns'] = (merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['SP 500 Value'] + merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['SP500 Tot Div']).cumsum()

# Aggregate Ticker returns across all positions.
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'Cum Stock Returns'] = (merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Ticker Share Value'] + merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Dividend Amt']).cumsum()

# Current position yield based on latest dividend and latest share price.
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'Market Yield'] = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Latest Div Amt'] / merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Ticker Adj Close']

# Current yield based on original cost basis for stock and latest dividend amount.
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'Current Yield'] = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Latest Div Amt'] / merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Unit Cost']

# Total SP returns pct.
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'Cum SP Returns Pct'] = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Cum Total SP Returns'] / merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Cum Invst'] - 1

# Total Tickers returns pct.
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'Cum Ticker Returns Pct'] = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'Cum Stock Returns'] / merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.loc[:, 'Cum Invst'] - 1

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.head(12)

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,...,Total SP500 Gain / (Loss),Total Stock Return,Total SP500 Return,Share Yield,Cum Total SP Returns,Cum Stock Returns,Market Yield,Current Yield,Cum SP Returns Pct,Cum Ticker Returns Pct
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,0.0,0,...,1604.553493,0.186935,0.670520,0.126306,3997.553493,2840.335365,0.010323,0.010949,0.670520,0.186935
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,0.0,1,...,386.635034,-0.384074,0.670520,0.007211,4960.808527,3195.490649,0.047385,0.028844,0.670520,0.076060
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,0.0,2,...,345.228125,0.953637,0.551923,0.000000,5931.536651,4417.490676,0.000000,0.000000,0.649886,0.228746
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,0.0,3,...,272.070048,-0.006971,0.670520,0.038125,6609.366700,4820.422245,0.012470,0.011907,0.651978,0.204840
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,0.0,4,...,3092.168362,0.240830,0.670520,0.158698,14313.135061,10542.632925,0.015065,0.016302,0.661906,0.224111
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,0.0,5,...,429.856729,0.618318,0.670520,0.295590,15384.071791,11580.104191,0.031963,0.042279,0.662503,0.251422
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,0.0,6,...,4258.067908,0.732765,0.670520,0.129785,25992.539699,22583.852888,0.021841,0.035011,0.665766,0.447315
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,0.0,7,...,1870.442105,0.723580,0.659201,0.129097,30700.421804,27474.407865,0.021841,0.034826,0.664755,0.489822


In [122]:
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 0 to 7
Data columns (total 59 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Acquisition Date             8 non-null      datetime64[ns]
 1   Position                     8 non-null      object        
 2   Ticker                       8 non-null      object        
 3   Quantity                     8 non-null      int64         
 4   Unit Cost                    8 non-null      float64       
 5   Cost Basis                   8 non-null      float64       
 6   Start of Year                8 non-null      datetime64[ns]
 7   Target_Alloc                 8 non-null      float64       
 8   Comments                     8 non-null      float64       
 9   Counts                       8 non-null      int64         
 10  Ticker #                     8 non-null      object        
 11  Latest Date                  8 non-null      date

In [123]:
# Subsetted df that will be used for the summary table on the dashboard.

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs[['Ticker #', 'Acquisition Date',
                                                                                                         'Ticker', 'Unit Cost', 'Cost Basis', 'Ticker Adj Close',
                                                                                                         'Ticker Return','SP Return', 'Stock Gain / (Loss)'
                                                                                                         ,'Share YTD', 'Closing High Adj Close'
                                                                                                         ,'Closing High Adj Close Date'
                                                                                                         ,'Pct off High', 'Dividend Amt', 'Share Yield'
                                                                                                                             ,'Market Yield', 'Current Yield'
                                                                                                                             ,'Ex-Div. Date', 'Latest Div Amt']]

# Converting to date time and coerce will force an out of bounds date to NaT.
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.loc[:, 'Ex-Div. Date'] = pd.to_datetime(merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.loc[:, 'Ex-Div. Date'], errors='coerce')

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary[0:10]

/home/rootdaenib/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Ticker #,Acquisition Date,Ticker,Unit Cost,Cost Basis,Ticker Adj Close,Ticker Return,SP Return,Stock Gain / (Loss),Share YTD,Closing High Adj Close,Closing High Adj Close Date,Pct off High,Dividend Amt,Share Yield,Market Yield,Current Yield,Ex-Div. Date,Latest Div Amt
0,AGG 0,2014-04-21,AGG,95.72,2393.00,101.523415,0.060629,0.551923,145.085365,0.023509,102.112717,2019-03-28,-0.005771,302.250000,0.126306,0.010323,0.010949,2019-04-01,1.04800
1,DBC 1,2014-04-21,DBC,26.21,576.62,15.954422,-0.391285,0.551923,-225.622716,0.118703,26.155830,2014-06-20,-0.390024,4.158000,0.007211,0.047385,0.028844,2018-12-24,0.75600
2,IAU 2,2014-04-21,IAU,12.51,625.50,24.440001,0.953637,0.551923,596.500027,-0.005696,26.360001,2016-07-08,-0.072838,0.000000,0.000000,0.000000,0.000000,NaT,0.00000
3,IGOV 3,2014-04-21,IGOV,50.72,405.76,48.432762,-0.045095,0.551923,-18.297903,0.005570,51.578995,2018-03-26,-0.060998,15.469472,0.038125,0.012470,0.011907,2018-12-18,0.60394
4,VEU 4,2014-04-21,VEU,43.92,4611.60,47.527245,0.082132,0.551923,378.760680,0.138579,52.347305,2018-01-26,-0.092078,731.850000,0.158698,0.015065,0.016302,2019-03-25,0.71600
5,VNQ 5,2014-04-21,VNQ,58.28,641.08,77.088570,0.322728,0.551923,206.894266,0.154636,79.396973,2019-04-12,-0.029074,189.497000,0.295590,0.031963,0.042279,2019-03-28,2.46400
6,VTI 6,2014-04-21,VTI,88.20,6350.40,141.382843,0.602980,0.551923,3829.164697,0.167998,141.990143,2018-09-20,-0.004277,824.184000,0.129785,0.021841,0.035011,2019-03-25,3.08800
7,VTI 7,2014-05-05,VTI,88.67,2837.44,141.382843,0.594483,0.541408,1686.810977,0.167998,141.990143,2018-09-20,-0.004277,366.304000,0.129097,0.021841,0.034826,2019-03-25,3.08800


In [124]:
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 0 to 7
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Ticker #                     8 non-null      object        
 1   Acquisition Date             8 non-null      datetime64[ns]
 2   Ticker                       8 non-null      object        
 3   Unit Cost                    8 non-null      float64       
 4   Cost Basis                   8 non-null      float64       
 5   Ticker Adj Close             8 non-null      float64       
 6   Ticker Return                8 non-null      float64       
 7   SP Return                    8 non-null      float64       
 8   Stock Gain / (Loss)          8 non-null      float64       
 9   Share YTD                    8 non-null      float64       
 10  Closing High Adj Close       8 non-null      float64       
 11  Closing High Adj Close Date  8 non-null      date

In [125]:
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.round(3)

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.head()

,Ticker #,Acquisition Date,Ticker,Unit Cost,Cost Basis,Ticker Adj Close,Ticker Return,SP Return,Stock Gain / (Loss),Share YTD,Closing High Adj Close,Closing High Adj Close Date,Pct off High,Dividend Amt,Share Yield,Market Yield,Current Yield,Ex-Div. Date,Latest Div Amt
0,AGG 0,2014-04-21,AGG,95.72,2393.00,101.523,0.061,0.552,145.085,0.024,102.113,2019-03-28,-0.006,302.250,0.126,0.010,0.011,2019-04-01,1.048
1,DBC 1,2014-04-21,DBC,26.21,576.62,15.954,-0.391,0.552,-225.623,0.119,26.156,2014-06-20,-0.390,4.158,0.007,0.047,0.029,2018-12-24,0.756
2,IAU 2,2014-04-21,IAU,12.51,625.50,24.440,0.954,0.552,596.500,-0.006,26.360,2016-07-08,-0.073,0.000,0.000,0.000,0.000,NaT,0.000
3,IGOV 3,2014-04-21,IGOV,50.72,405.76,48.433,-0.045,0.552,-18.298,0.006,51.579,2018-03-26,-0.061,15.469,0.038,0.012,0.012,2018-12-18,0.604
4,VEU 4,2014-04-21,VEU,43.92,4611.60,47.527,0.082,0.552,378.761,0.139,52.347,2018-01-26,-0.092,731.850,0.159,0.015,0.016,2019-03-25,0.716


In [126]:
# Re-formatting the % columns.

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Ticker Return'] = pd.Series(["{0:.1f}%".format(val * 100) 
                                                                                     for val in merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Ticker Return']], index = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.index)

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Share YTD'] = pd.Series(["{0:.1f}%".format(val * 100) 
                                                                                     for val in merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Share YTD']], index = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.index)

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['SP Return'] = pd.Series(["{0:.1f}%".format(val * 100) 
                                                                                     for val in merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['SP Return']], index = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.index)


merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Pct off High'] = pd.Series(["{0:.1f}%".format(val * 100) 
                                                                                     for val in merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Pct off High']], index = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.index)

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Share Yield'] = pd.Series(["{0:.1f}%".format(val * 100) 
                                                                                     for val in merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Share Yield']], index = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.index)

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Market Yield'] = pd.Series(["{0:.1f}%".format(val * 100) 
                                                                                     for val in merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Market Yield']], index = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.index)

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Current Yield'] = pd.Series(["{0:.1f}%".format(val * 100) 
                                                                                     for val in merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Current Yield']], index = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.index)

In [127]:
# https://plot.ly/python/table/#use-a-pandas-dataframe

trace = go.Table(
    header=dict(values=list(merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.columns),
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Ticker #'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Acquisition Date'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Ticker'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Unit Cost'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Cost Basis'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Ticker Adj Close'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Ticker Return'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['SP Return'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Stock Gain / (Loss)'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Share YTD'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Closing High Adj Close'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Closing High Adj Close Date'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Pct off High'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Dividend Amt'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Share Yield'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Market Yield'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Current Yield'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Ex-Div. Date'],
                       merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary['Latest Div Amt']
                      ],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5,
                height = 40))

data = [trace] 
iplot(data)

## Data Outputs

In [128]:
# Generate the base file that will be used for Dash dashboard.

# merged_portfolio_sp_latest_YTD_sp_closing_high.to_csv('analyzed_portfolio.csv')

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs.to_csv('analyzed_portfolio.csv')

In [129]:
merged_portfolio_sp_latest_YTD_sp_closing_high.head()

merged_portfolio_sp_latest_YTD_sp_closing_high_tickers = merged_portfolio_sp_latest_YTD_sp_closing_high[['Ticker']]

merged_portfolio_sp_latest_YTD_sp_closing_high_tickers = merged_portfolio_sp_latest_YTD_sp_closing_high_tickers.drop_duplicates(['Ticker'], keep='first')

merged_portfolio_sp_latest_YTD_sp_closing_high_tickers = merged_portfolio_sp_latest_YTD_sp_closing_high_tickers['Ticker'].unique()

merged_portfolio_sp_latest_YTD_sp_closing_high_tickers = merged_portfolio_sp_latest_YTD_sp_closing_high_tickers.tolist()

# Appending SPY for comparisons to existing positions in initial interactive chart.
tickers_append = ['SPY']

merged_portfolio_sp_latest_YTD_sp_closing_high_tickers.extend(tickers_append)

merged_portfolio_sp_latest_YTD_sp_closing_high_tickers = pd.DataFrame(data=merged_portfolio_sp_latest_YTD_sp_closing_high_tickers, columns=['Ticker'])

merged_portfolio_sp_latest_YTD_sp_closing_high_tickers.sort_values(by='Ticker', ascending=True, inplace=True)

merged_portfolio_sp_latest_YTD_sp_closing_high_tickers.head()

merged_portfolio_sp_latest_YTD_sp_closing_high_tickers.to_csv('tickers.csv')

In [130]:
# Generate the file for the comparison of target % allocation relative to actual allocation.
merged_portfolio_sp_latest_YTD_sp_contr_subset_pivot.to_csv('allocation.csv')

In [131]:
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_summary.to_csv('data_summary.csv', index=False)

In [132]:
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_all_payers = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs[merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Dividend Amt']>0]

# merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_all_payers.head()

merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_all_payers.to_csv('analyzed_portfolio_divs_only.csv', index=False)

## Dividend Visualizations

In [133]:
merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs[0:10]

,Acquisition Date,Position,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year,Target_Alloc,Comments,Counts,...,Total SP500 Gain / (Loss),Total Stock Return,Total SP500 Return,Share Yield,Cum Total SP Returns,Cum Stock Returns,Market Yield,Current Yield,Cum SP Returns Pct,Cum Ticker Returns Pct
0,2014-04-21,iShares Core US Aggregate Bond ETF (AGG),AGG,25,95.72,2393.00,2018-12-31,0.127,0.0,0,...,1604.553493,0.186935,0.670520,0.126306,3997.553493,2840.335365,0.010323,0.010949,0.670520,0.186935
1,2014-04-21,Invesco DB Commodity Tracking (DBC),DBC,22,26.21,576.62,2018-12-31,0.033,0.0,1,...,386.635034,-0.384074,0.670520,0.007211,4960.808527,3195.490649,0.047385,0.028844,0.670520,0.076060
2,2014-04-21,iShares Gold Trust (IAU),IAU,50,12.51,625.50,2018-12-31,0.033,0.0,2,...,345.228125,0.953637,0.551923,0.000000,5931.536651,4417.490676,0.000000,0.000000,0.649886,0.228746
3,2014-04-21,iShares International Treasury Bond ETF (IGOV),IGOV,8,50.72,405.76,2018-12-31,0.023,0.0,3,...,272.070048,-0.006971,0.670520,0.038125,6609.366700,4820.422245,0.012470,0.011907,0.651978,0.204840
4,2014-04-21,Vanguard FTSE All-Wld ex-US ETF (VEU),VEU,105,43.92,4611.60,2018-12-31,0.250,0.0,4,...,3092.168362,0.240830,0.670520,0.158698,14313.135061,10542.632925,0.015065,0.016302,0.661906,0.224111
5,2014-04-21,Vanguard Real Estate ETF (VNQ),VNQ,11,58.28,641.08,2018-12-31,0.034,0.0,5,...,429.856729,0.618318,0.670520,0.295590,15384.071791,11580.104191,0.031963,0.042279,0.662503,0.251422
6,2014-04-21,Vanguard Total Stock Market ETF (VTI),VTI,72,88.20,6350.40,2018-12-31,0.500,0.0,6,...,4258.067908,0.732765,0.670520,0.129785,25992.539699,22583.852888,0.021841,0.035011,0.665766,0.447315
7,2014-05-05,Vanguard Total Stock Market ETF (VTI),VTI,32,88.67,2837.44,2018-12-31,0.500,0.0,7,...,1870.442105,0.723580,0.659201,0.129097,30700.421804,27474.407865,0.021841,0.034826,0.664755,0.489822


In [134]:
# Compare total shareholder return for each position relative to SP500.

trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_all_payers['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_all_payers['Total Stock Return'],
    name = 'Ticker TSR')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_all_payers['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs_all_payers['Total SP500 Return'],
    name = 'SP500 TSR')
    
data = [trace1, trace2]

layout = go.Layout(title = 'TSR Comparison, Ticker versus SP500'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".1%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1.2)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [135]:
trace1 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Cum Invst'],
    mode = 'lines+markers',
    name = 'Cum Invst')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Cum Stock Returns'],
    mode = 'lines+markers',
    name = 'Cum Stock Returns')

trace3 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Cum Total SP Returns'],
    mode = 'lines+markers',
    name = 'Cum SP500 Returns')

data = [trace1, trace2, trace3]

layout = go.Layout(title = 'TSR Comparisons by Ticker'
    , barmode = 'group'
    , yaxis=dict(title='Returns')
    , xaxis=dict(title='Ticker')
    , legend=dict(x=1,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [136]:
# Graphing the total return percents.

trace1 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Cum Ticker Returns Pct'],
    mode = 'lines+markers',
    name = 'Cum Ticker Returns Pct')

trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Ticker #'],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high_incl_divs['Cum SP Returns Pct'],
    mode = 'lines+markers',
    name = 'Cum SP Returns Pct')

data = [trace1, trace2]

layout = go.Layout(title = 'TSR Pct Comparisons by Ticker'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".1%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=1,y=1)
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)